In [1]:
import numpy
import pandas as pd

In [2]:
df = pd.read_csv("pima-indians-diabetes.csv",header=0,infer_datetime_format=True)
df.shape

(767, 9)

In [3]:
df.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [4]:
df = df.iloc[:,0:9]
df.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [5]:
df.tail()

,6,148,72,35,0,33.6,0.627,50,1
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1
766,1,93,70,31,0,30.4,0.315,23,0


In [6]:
dataset=numpy.array(df)
dataset

array([[1.00e+00, 8.50e+01, 6.60e+01, ..., 3.51e-01, 3.10e+01, 0.00e+00],
       [8.00e+00, 1.83e+02, 6.40e+01, ..., 6.72e-01, 3.20e+01, 1.00e+00],
       [1.00e+00, 8.90e+01, 6.60e+01, ..., 1.67e-01, 2.10e+01, 0.00e+00],
       ...,
       [5.00e+00, 1.21e+02, 7.20e+01, ..., 2.45e-01, 3.00e+01, 0.00e+00],
       [1.00e+00, 1.26e+02, 6.00e+01, ..., 3.49e-01, 4.70e+01, 1.00e+00],
       [1.00e+00, 9.30e+01, 7.00e+01, ..., 3.15e-01, 2.30e+01, 0.00e+00]])

# 1- Parti Boyutunu ve Dönem Sayısı Nasıl Ayarlanır

In [7]:
# Epochs ve batch size'ı ızgarada aramak için scikit-learn'i kullanalım.
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [8]:
# KerasClassifier için gerekli olan modeli oluşturma 
def create_model():
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# model oluşturma
model = KerasClassifier(build_fn=create_model, verbose=0)
# Izgara arama parametrelerini tanımladık.
batch_size = [2, 4, 8, 16, 32, 64, 128]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.724934 using {'batch_size': 2, 'epochs': 100}
0.674071 (0.012885) with: {'batch_size': 2, 'epochs': 10}
0.693597 (0.008566) with: {'batch_size': 2, 'epochs': 50}
0.724934 (0.018805) with: {'batch_size': 2, 'epochs': 100}
0.642754 (0.040215) with: {'batch_size': 4, 'epochs': 10}
0.674020 (0.020103) with: {'batch_size': 4, 'epochs': 50}
0.692315 (0.014912) with: {'batch_size': 4, 'epochs': 100}
0.598478 (0.028202) with: {'batch_size': 8, 'epochs': 10}
0.687051 (0.026068) with: {'batch_size': 8, 'epochs': 50}
0.690993 (0.006892) with: {'batch_size': 8, 'epochs': 100}
0.589195 (0.091055) with: {'batch_size': 16, 'epochs': 10}
0.618010 (0.014625) with: {'batch_size': 16, 'epochs': 50}
0.658369 (0.021920) with: {'batch_size': 16, 'epochs': 100}
0.488843 (0.041413) with: {'batch_size': 32, 'epochs': 10}
0.625868 (0.034619) with: {'batch_size': 32, 'epochs': 50}
0.680545 (0.015360) with: {'batch_size': 32, 'epochs': 100}
0.591876 (0.048346) with: {'batch_size': 64, 'epochs': 10}
0.6336

# 2- Eğitim Optimizasyon Algoritmasını Ayarlama

In [9]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [10]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(optimizer='adam'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.689701 using {'optimizer': 'Nadam'}
0.607496 (0.042194) with: {'optimizer': 'SGD'}
0.681878 (0.017545) with: {'optimizer': 'RMSprop'}
0.593178 (0.031986) with: {'optimizer': 'Adagrad'}
0.460069 (0.143152) with: {'optimizer': 'Adadelta'}
0.650495 (0.051606) with: {'optimizer': 'Adam'}
0.637500 (0.026708) with: {'optimizer': 'Adamax'}
0.689701 (0.017545) with: {'optimizer': 'Nadam'}


# 3- Öğrenme Hızını ve Momentum Ayarlama

In [11]:
# Öğrenme hızını ve momentumu ızgarada aramak için scikit-learn'i kullanalım
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [12]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(learn_rate=0.01, momentum=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
learn_rate = [0.001, 0.005, 0.0015, 0.01, 0.025]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.688404 using {'learn_rate': 0.001, 'momentum': 0.4}
0.638756 (0.053893) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.661024 (0.004297) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.688404 (0.014910) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.680520 (0.034070) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.657138 (0.018936) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.659768 (0.034610) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.675357 (0.022334) with: {'learn_rate': 0.005, 'momentum': 0.0}
0.646686 (0.006024) with: {'learn_rate': 0.005, 'momentum': 0.2}
0.671421 (0.030722) with: {'learn_rate': 0.005, 'momentum': 0.4}
0.650618 (0.024354) with: {'learn_rate': 0.005, 'momentum': 0.6}
0.650618 (0.024354) with: {'learn_rate': 0.005, 'momentum': 0.8}
0.649316 (0.026145) with: {'learn_rate': 0.005, 'momentum': 0.9}
0.672743 (0.007691) with: {'learn_rate': 0.0015, 'momentum': 0.0}
0.668816 (0.012712) with: {'learn_rate': 0.0015, 'momentum': 0.2}
0.634916 (0.013598) with: {'

# 4- Ağ Ağırlığı Başlatma Nasıl Ayarlanır

In [15]:
# Ağırlık başlatmayı ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [16]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(init_mode='uniform'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.730127 using {'init_mode': 'normal'}
0.713189 (0.020960) with: {'init_mode': 'uniform'}
0.705362 (0.009254) with: {'init_mode': 'lecun_uniform'}
0.730127 (0.016725) with: {'init_mode': 'normal'}
0.651920 (0.022571) with: {'init_mode': 'zero'}
0.702798 (0.034055) with: {'init_mode': 'glorot_normal'}
0.715804 (0.018833) with: {'init_mode': 'glorot_uniform'}
0.694924 (0.022874) with: {'init_mode': 'he_normal'}
0.688378 (0.010312) with: {'init_mode': 'he_uniform'}


# 5- Nöron Aktivasyon Fonksiyonu Nasıl Ayarlanır

In [17]:
# Aktivasyon işlevini ızgarada aramak için scikit-learn'i kullanın
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [18]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(activation='relu'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.732746 using {'activation': 'softplus'}
0.651920 (0.022571) with: {'activation': 'softmax'}
0.732746 (0.029451) with: {'activation': 'softplus'}
0.692341 (0.018176) with: {'activation': 'softsign'}
0.721007 (0.023967) with: {'activation': 'relu'}
0.668847 (0.004310) with: {'activation': 'tanh'}
0.701455 (0.020948) with: {'activation': 'sigmoid'}
0.687117 (0.013365) with: {'activation': 'hard_sigmoid'}
0.719736 (0.026301) with: {'activation': 'linear'}


# 6- Bırakma Düzenlemesini Ayarlama

In [19]:
# Bırakma oranını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [20]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(dropout_rate=0.0, weight_constraint=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
#Izgara arama parametrelerini tanımladık.
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.724949 using {'dropout_rate': 0.6, 'weight_constraint': 4}
0.705351 (0.003148) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.722304 (0.010808) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.693607 (0.002412) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.719705 (0.010759) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.709329 (0.039449) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.694935 (0.014216) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.713205 (0.020035) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.701460 (0.026469) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.700143 (0.007539) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.717121 (0.022612) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.698856 (0.016060) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.724944 (0.022626) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.715829 (0.029708) with: {'dropout_rate': 0.2, 'weight_constraint': 

# 7- Gizli Katmandaki Nöron Sayısı Nasıl Ayarlanır

In [21]:
# Nöron sayısını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [22]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(neurons=1):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.717111 using {'neurons': 15}
0.706659 (0.013702) with: {'neurons': 1}
0.701445 (0.006122) with: {'neurons': 5}
0.700158 (0.028309) with: {'neurons': 10}
0.717111 (0.017091) with: {'neurons': 15}
0.710621 (0.033314) with: {'neurons': 20}
0.717096 (0.009275) with: {'neurons': 25}
0.710580 (0.011015) with: {'neurons': 30}
